In [ ]:
usage = """Run with papermill:
     
papermill substitution_error_rate_report.ipynb output_substitution_error_rate_report.ipynb \
    -p h5_substitution_error_rate <> \
    -p png_substitution_error_rate_by_mut_type_and_source <> \
    -p png_substitution_error_rate_asymmetry <> \
    -p png_substitution_error_rate_by_motif_thresh0 <> \
    -p png_substitution_error_rate_by_motif_thresh3 <> \
    -p png_substitution_error_rate_by_motif_thresh5 <> \
    -p png_substitution_error_rate_by_motif_thresh10 <> \
    -p png_positional_substitution_error_rate <>



Then convert to html

jupyter nbconvert --to html output_substitution_error_rate_report.ipynb --template classic --no-input --output substitution_error_rate_report.html"""

In [ ]:
import pandas as pd
import os
from IPython.display import Image, HTML

pd.options.display.max_rows = 200

In [ ]:
# papermill parameters
h5_substitution_error_rate = None
png_substitution_error_rate_by_mut_type_and_source = None
png_substitution_error_rate_asymmetry = None
png_substitution_error_rate_by_motif_thresh0 = None
png_substitution_error_rate_by_motif_thresh3 = None
png_substitution_error_rate_by_motif_thresh5 = None
png_substitution_error_rate_by_motif_thresh10 = None
png_positional_substitution_error_rate = None

In [ ]:
# check that we got all the inputs
missing = list()
for varname in [
    "h5_substitution_error_rate",
    "png_substitution_error_rate_by_mut_type_and_source",
    "png_substitution_error_rate_asymmetry",
    "png_substitution_error_rate_by_motif_thresh0",
    "png_substitution_error_rate_by_motif_thresh3",
    "png_substitution_error_rate_by_motif_thresh5",
    "png_substitution_error_rate_by_motif_thresh10",
]:
    if locals()[varname] is None:
        missing.append(varname)

if len(missing) > 0:
    raise ValueError(f"Following inputs missing:\n{(os.linesep).join(missing)}")

In [ ]:
def revcomp(seq):
    """Reverse complements DNA given as string

    Parameters
    ----------
    seq: Union[str,list,np.ndarray]
        DNA string

    Returns
    -------
    str | list | np.ndarray
    """
    complement = {
        "A": "T",
        "C": "G",
        "G": "C",
        "T": "A",
        "a": "t",
        "c": "g",
        "g": "c",
        "t": "a",
    }
    if isinstance(seq, str):
        reverse_complement = "".join(
            complement.get(base, base) for base in reversed(seq)
        )
    elif isinstance(seq, list):
        reverse_complement = [complement.get(base, base) for base in reversed(seq)]
    elif isinstance(seq, np.ndarray):
        reverse_complement = np.array(
            [complement.get(base, base) for base in reversed(seq)]
        )

    return reverse_complement


def create_matched_forward_and_reverse_strand_errors_dataframe(df_motifs):
    df_motifs = df_motifs.astype({"ref_motif": str, "alt_motif": str})
    df_motifs.loc[:, "mut_type"] = (
        df_motifs["ref_motif"].str.slice(1, 2)
        + "->"
        + df_motifs["alt_motif"].str.slice(1, 2)
    )
    df_for = df_motifs[(df_motifs["ref"] == "C") | (df_motifs["ref"] == "T")].copy()
    df_rev = df_motifs[(df_motifs["ref"] == "A") | (df_motifs["ref"] == "G")].copy()
    df_rev.loc[:, "ref_motif"] = df_rev["ref_motif"].apply(revcomp)
    df_rev.loc[:, "alt_motif"] = df_rev["alt_motif"].apply(revcomp)
    df_rev = df_rev.set_index(["ref_motif", "alt_motif"])
    df_for = df_for.set_index(["ref_motif", "alt_motif"])
    df_err = df_for.filter(regex="mut_type|error_rate|snp_count").join(
        df_rev.filter(regex="error_rate|snp_count"), lsuffix="_f", rsuffix="_r"
    )
    for c in df_err.filter(regex="error[\w]+_f").columns:
        df_err.loc[:, c[:-2]] = df_err.filter(regex=c[:-1]).mean(axis=1)
    for c in df_err.filter(regex="snp_count_[\w]+_f").columns:
        df_err.loc[:, c[:-2]] = df_err.filter(regex=c[:-1]).sum(axis=1)
    return df_err

In [ ]:
df = pd.read_hdf(h5_substitution_error_rate, key="motif_1")

df_err = create_matched_forward_and_reverse_strand_errors_dataframe(df)

In [ ]:
display(HTML('<font size="6">Substitution error rate report</font>'))

# Introduction 

This report contains an analysis of the substitution error rates in a sample sequenced on an Ultima Genomics sequencer. Only substitution error rates are considered, indels are not. Errors are analyzed with respect to the reference and alternative bases, trinucleotide context, cycle-skip status and position on the read.  

# Results 

## Average substitution error rates 

In [ ]:
df_tmp = df.filter(regex="error_rate").mean().rename("Rate of substitution errors").to_frame()
df_tmp.index = ["All motifs, log likelihood thresh=0", "All motifs, log likelihood thresh=3", "All motifs, log likelihood thresh=5", "Cycle-skip motifs, log likelihood thresh=10"]
df_tmp2 = df.filter(regex="snp_count").sum().rename("Number of substitution errors").to_frame()
df_tmp2.index = ["All motifs, log likelihood thresh=0", "All motifs, log likelihood thresh=3", "All motifs, log likelihood thresh=5", "Cycle-skip motifs, log likelihood thresh=10"]
df_tmp = df_tmp.join(df_tmp2)

df_ncskp = df_err[df_err["error_rate_bq10"].isnull()].filter(regex="bq[0-9]$")
df_ncskp = df_ncskp.agg({c: "sum" if "count" in c else "mean" for c in df_ncskp.columns})
df_cskp = df_err[~df_err["error_rate_bq10"].isnull()].filter(regex="bq10$")
df_cskp = df_cskp.agg({c: "sum" if "count" in c else "mean" for c in df_cskp.columns})

df_agg = pd.concat((df_ncskp, df_cskp))
df_count = df_agg.filter(regex="snp").rename("Number of substitution errors")
df_count.index = ["Non cycle-skip motifs, log likelihood thresh=0", "Non cycle-skip motifs, log likelihood thresh=3", "Non cycle-skip motifs, log likelihood thresh=5", "Cycle-skip motifs"]
df_rate = df_agg.filter(regex="error_rate").rename("Rate of substitution errors")
df_rate.index = ["Non cycle-skip motifs, log likelihood thresh=0", "Non cycle-skip motifs, log likelihood thresh=3", "Non cycle-skip motifs, log likelihood thresh=5", "Cycle-skip motifs"]

best_motif = df_err.sort_values("error_rate_bq5")[
    ["error_rate_bq5", "snp_count_bq5"]
].head(1)
best_motif.index = [f"Lowest error motif: {x[0]}→{x[1]}" for x in best_motif.index]
worst_motif = df_err.sort_values("error_rate_bq5")[
    ["error_rate_bq5", "snp_count_bq5"]
].tail(1)
worst_motif.index = [f"Highest error motif: {x[0]}→{x[1]}" for x in worst_motif.index]
motifs = pd.concat((best_motif, worst_motif))
motifs.columns = ["Rate of substitution errors", "Number of substitution errors"]

df_ref_alt = df_err[["error_rate_bq10", "snp_count_bq10"]].dropna()
df_ref_alt.index = [f"{x[0][1]}→{x[1][1]} (cycle-skip motifs)" for x in df_ref_alt.index]
df_ref_alt = df_ref_alt.groupby(level=0).agg({"error_rate_bq10": "mean", "snp_count_bq10": "sum"})
df_ref_alt.columns = ["Rate of substitution errors", "Number of substitution errors"]

pd.concat(
    (
        motifs,
        df_ref_alt,
        df_tmp.loc[["All motifs, log likelihood thresh=5"]],
        df_rate.to_frame()
        .join(df_count)
        .loc[["Non cycle-skip motifs, log likelihood thresh=5", "Cycle-skip motifs"]],
    )
).iloc[::-1].style.format("{:.1e}")

## Substitution error rate by mutation type and noise source

In [ ]:
Image(png_substitution_error_rate_by_mut_type_and_source, width=1000)

Substitution error rate due to sequencing errors at different filtering stringencies, and due to non-sequencing errors. The error rate is calculated for log-likelihood thresholds of 0 (no filtering), 3 (Phred 30) and 5 (Phred 50). Additionally, they are calculated for cycle-skip motifs only by setting a log likelihood threshold of 10 (Phred 100). The errors are grouped by the mutation type - reference and alternative bases, then the cycle-skip error rate is subtracted from the other errors rates (0, 3, 5 thresholds) to obtain an estimate of the sequencing errors. 

## Detailed error rate profile by trinucleotide context 

Substitution errors in the plots below were grouped by mutation type (reference and alternative base) and trinucleotide context, and filtered by varying likelihood thresholds. Each pair of bars represents the error rate in a specific trinucleotide context (e.g. $TCA$) when the central base ($T\boldsymbol CA$) is measured as a specific alternative base. The color represents the 6 different mutation types. Reverse complement errors are grouped together where the forward error (corresponding to the annotation, e.g. TCA->TAA) is shown in color (blue for C->A) and the respective reverse error (TGA->TTA) is shown in black.

Cycle-skip motifs are annotated by a + before the trinucleotide context on the x axis and by a seperate color.

The legend shows the mean and median error for both cycle-skip motifs only (cskp) and for all the motifs (all). The metrics are unweighted so all motifs are represented equally without taking genomic abundance into account.

In [ ]:
Image(png_substitution_error_rate_by_motif_thresh0)

In [ ]:
Image(png_substitution_error_rate_by_motif_thresh3)

In [ ]:
Image(png_substitution_error_rate_by_motif_thresh5)

In [ ]:
Image(png_substitution_error_rate_by_motif_thresh10)

Note that in the last plot only cycle-skip motifs are shown, as other motifs cannot reach a likelihood threshold of 10.

## Substitution error rate as a function of position 

In [ ]:
Image(png_positional_substitution_error_rate)

Cycle-skip substitutions are binned by their position on the read, and grouped by mutation type. For each mutation type the forward and reverse errors are shown. The profile are generally expected to be position independent as they are unrelated to sequencing, so any deviation from a uniform profile can either point to an artefact or assist in the reduction of substitution error rates in relevant protocols. Most notably, increased error rates in the beginning of reads can indicate either improper trimming of adapters or UMIs, or specific steps in the preparation protocols. Additionally, for cfDNA samples the behaviour around 170-200bp where there is a transition for mono-nucleosome to di-nucleosome could be of interest. 

## Asymmetry in cycle-skip errors

In [ ]:
Image(png_substitution_error_rate_asymmetry, width = 400)

Asymmtery in substitution error rates of cycle-skip motifs. For each mutation type the ratio of forward and reverse errors is measured per trinucleotide context, and shown both as a boxplot of the distribution and as a simple mean. The annotation of the mean value indicates the magnitude of bias in linear scale, while the x axis is in a log2.
Asymmetry is generally expected in sequencing errors, but in non-sequencing errors it can point to specific mechanisms driving it.

# Methods 

## Definition of a substitution error

An event is defined as a substitution error if the following conditions are met:
1. A substitution from the reference genome is reported by the aligner
2. Only one read presents a substitution in this locus, out of a large enough number of reads mapped to that locus (20 by default but may vary, especially for low coverage samples). This condition guarantees that germline variants are extremely unlikely to be counted as errors.
3. A few adjacent bases (5 by default but may vary) on either end of the substitution match the reference genome precisely. This condition makes sure only true substitutions are counted, as it is often the case that homopolymer indels can be interpreted as substitutions by aligners in ambiguous cases. 


## Cycle-skip substitutions 

In flow based sequencing, each sequencing cycle is composed of four individual so-called flows, where a single nucleotide is introduced and the signal measured indicates the number of nucleotides incorporated, or the length of the homopolymer (hmer) of the respective base. For example, the sequence TGCTACAAAGGGGGC would be read this way in flow-based sequencing when the order in which nucleotides are introduced, aka the flow order, is TGCA:

Measured base:   T G C A T G C A T G C  A  T   G   C  
Sequence:        T G C   T   C A T     AAA   GGGGG C
Homopolymers:    1 1 1 0 1 0 1 1 1 0 0  3  0   5   1


A substitution of a single base yields a change in at least 2 homopolymers, and in some cases many more. Let us consider two examples: TGC->TTC and TGC->TAC


TGC:

Sequence:            T  G C

Measured base:   T  G C A T G C 

Homopolymers:   1  1 1


TTC:

Sequence:            TT   C

Measured base:   T  G C A T G C 

Homopolymers:   2  0 1 


TAC:

Sequence:            T         A       C

Measured base:   T  G C A T G C 

Homopolymers:   1  0  0 1 0 0 1 
 
 
  

In the first example (TGC->TTC) the first 2 homopolymers are changed, 1T->2T and 1G->0G, representing a change in 2 measured signals. Therefore, in order for such a substitution error (TGC->TTC) to occur two homopolymers would have to be miscalled. In the second example (TGC->TAC), the substitution leads to the C base being synthesized a full cycle later, leading to numerous changes in signal. Interestingly, this so-called cycle-skip causes the remainder of the signal to be unsynchronized, making the signal completely different so that sequencing error are extremely unlikely. About 42% of all substitutions are cycle-skip substitutions.

## Likelihood scores


For each substitution a log-likelihood score is calculated, by comparing a short local allele containing the substitution and the respective reference allele. By definition the substitution considered is the only difference between the alleles. Log-likelihood scores are usually capped at 10 (10 orders of magnitude), a value that is only possible for cycle-skip motifs* and the vast majority of cycle-skip motifs are above it.

\* The log likelihood score of non cycle-skip substitutions is the sum of log likelihoods over the 2 homopolymers where the alleles differ. A Phred score higher than 50 would be required for both homopolymers for achieve a score of 10 (equivalent to Phred 100), but in practice reported homopolymer indel are always higher.

## Attribution of error to a source 

A large part of the observed substitution error originates prior to sequencing, i.e. artefacts in the extraction and preparation protocol, errors incurred during sample storage, true somatic mutations that are misannotated as errors, or other unknown sources. These errors cannot be described by the standard likelihood model that only applies to sequencing errors, but their frequency can be estimated from their appearance in cycle-skip motifs, because the likelihood of sequencing errors in these motifs is negligible relative to other types of errors.